# **Trabajo Final de curso**
# **Método de Elementos Finitos con Software Libre**
***Alumno: Julián Esequiel Jurado***


---


En este trabajo se ha implementado un modelo tridimensional a escala
mesoscópica que permite estudiar la respuesta mecanica de homigones sometidos compresión uniaxial para un analisis elastico y lineal. Para el modelo se han discretizado dos fases: el agregado y la matriz de mortero. Se ha desarrollado un codigo en python que nos permite generar una distribución aleatoria de agregados circulares a partir de una curva de distribución de tamaños. Se ha empleado un modelo elastico lineal y se ha realizado su resolución por el metodo de elementos finitos utilizando la libreria de codigo abierto Fenics.


## Geometría y propiedades de los materiales

El trabajo se centra en la simulación numerica de  ensayos de compresión directa sobre probetas cilindricas de 100 mm  de diametro y una altura de 200mm. El ensayo consiste en aplicar una fuerza de compresión en la superficie superior del cilindro de hormigon. La Fig.1 muestra una imagen esquemática del modelo propuesto junto con las condiciones de borde que constan de la aplicación de un desplazamiento impuesto en la superficie superior y la restricción de los desplazamientos en la superficie inferior.

### Código en FEniCS

Ejecutar este documento en forma dinámica: [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rirastorza/Intro2FEM/blob/master/Elementos_finitos_en_2D/termico2D.ipynb)


Para correr en google colab ejecutar la siguientes instrucciones (https://fem-on-colab.github.io/).


In [1]:
%%capture
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


A partir del archivo con la extensión .msh. Lo debemos importar a FEniCS, una forma de hacerlo es transformando la malla a formato xml. Para esto usamos la función dolfin-convert. Ejecutando en consola:

In [3]:
%%bash
dolfin-convert cylinder.msh cylinder.xml

Converting from Gmsh format (.msh, .gmsh) to DOLFIN XML format
Expecting 167299 vertices
Found all vertices
Expecting 972256 cells
Found all cells
Conversion done


Se generan tres archivos: cylinder.xml, cylinder_physical_region.xml, y cylinder_facet_region.xml, que contienen la malla, las marcas de las regiones físicas, y las marcas de las fronteras o bordes.



Continuamos ahora desarrollando el código, con el enfoque de un modelo 3D.

In [4]:
from fenics import *
from dolfin import *

mesh = Mesh("cylinder.xml");
subdomains = MeshFunction('size_t',mesh,"cylinder_physical_region.xml");
boundary_markers = MeshFunction('size_t',mesh,"cylinder_facet_region.xml");

Luego definimos la función que nos calcula la deformación, utilizando una función simbólica de Python:

In [5]:
def eps(v):
    return sym(grad(v))

Ahora definimos los parámetros de Lamé.

In [6]:
E = Constant(1e5)
nu = Constant(0.3)
mu = E/2/(1+nu)
lmbda = E*nu/(1+nu)/(1-2*nu)

Y la expresión de la tensión en base a la ley de Hooke para medios isotrópicos.

In [7]:
def sigma(v):
    return lmbda*tr(eps(v))*Identity(3) + 2.0*mu*eps(v)

Luego el espacio de funciones ***$V$*** (función de espacio vectorial) y la condición de borde en la superficie inferior.

In [8]:
#tol = 1E-14
#Constantes termicas
E_0 = Constant(70000.0)
E_1 = Constant(25000.0)

#Fuente, Temperatura variable
DeltaU = 4.0

V = VectorFunctionSpace(mesh, 'CG', 2)

#Defino condiciones de contorno por medio del archivo xml
bx0 = DirichletBC(V, Constant((0, 0, DeltaU)), boundary_markers,562)
bx1 = DirichletBC(V, Constant((0, 0, 0)), boundary_markers, 563)
bcs = [bx0,bx1]

class K(UserExpression):
    def __init__(self, subdomains, E_0, E_1, **kwargs):
        super().__init__(**kwargs)
        self.subdomains = subdomains
        self.E_0 = E_0
        self.E_1 = E_1
    def eval_cell(self, values, x, cell):
        if self.subdomains[cell.index] == 2:#capa interna del cilindro
            values[0] = self.E_1
        else:#capa externa del cilindro
            values[0] = self.E_0

kappa = K(subdomains, E_0, E_1, degree=2)


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_6d76beae4ab896e8ababfd518e901880eb8c9c6a

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00678158 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing representation of forms
INFO:FFC:  
INFO:FFC:Compiler stage 2 finished in 0.942261 seconds.

INFO:FFC:Compiler stage 3

La formulación variacional del problema es la siguiente:

$$ \int_{\Omega} \sigma \cdot \varepsilon_{v} \ dx - \int_{\partial \Omega} v \cdot \left(\sigma \cdot n\right) \ ds = \int_{\Omega} f\cdot v \ dx \tag{14} $$

si eliminamos la parte de condiciones de borde que no son Dirichlet queda:

$$ \int_{\Omega} \sigma \cdot \varepsilon_{v} \ dx = \int_{\Omega} f\cdot v \ dx \tag{16} $$

en código es:

In [9]:
#Formulación variacional
f = Constant((0, 0, 0))
u = TrialFunction(V)
v = TestFunction(V)
a = inner(sigma(u), eps(v))*dx
l = inner(f, v)*dx


Ahora resolvemos. Noten que aquí no lo hicimos con la forma bilineal y lineal, lo implementa internamente FEniCS.

In [10]:
#Solución
u = Function(V, name="Desplazamiento")
solve(a == l, u, bcs)

Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling form ffc_form_adbd1df009f5c63caa55deaadc71fe2a8feedea5

INFO:FFC:Compiler stage 1: Analyzing form(s)
INFO:FFC:-----------------------------------
DEBUG:FFC:  Preprocessing form using 'uflacs' representation family.
INFO:UFL_LEGACY:Adjusting missing element cell to tetrahedron.
INFO:UFL_LEGACY:Adjusting missing element cell to tetrahedron.
INFO:UFL_LEGACY:Adjusting missing element cell to tetrahedron.
INFO:UFL_LEGACY:Adjusting missing element cell to tetrahedron.
INFO:FFC:  
INFO:FFC:  Geometric dimension:       3
  Number of cell subdomains: 0
  Rank:                      1
  Arguments:                 '(v_0)'
  Number of coefficients:    1
  Coefficients:              '[f_16]'
  Unique elements:           'Vector<3 x CG2(?,?)>, Vector<3 x R0(?,?)>, Vector<3 x 
                             CG1(?,?)>'
  Unique sub elements:       'Vector<3 x CG2(?,?)>, Vector<3 x R0(?,?)>, Vector<3 x 
     

RuntimeError: 

*** -------------------------------------------------------------------------
*** DOLFIN encountered an error. If you are not able to resolve this issue
*** using the information listed below, you can ask for help at
***
***     https://fenicsproject.discourse.group/
***
*** Remember to include the error message listed below and, if possible,
*** include a *minimal* running example to reproduce the error.
***
*** -------------------------------------------------------------------------
*** Error:   Unable to successfully call PETSc function 'KSPSolve'.
*** Reason:  PETSc error code is: 76 (Error in external library).
*** Where:   This error was encountered inside /tmp/dolfin-src/dolfin/la/PETScKrylovSolver.cpp.
*** Process: 0
*** 
*** DOLFIN version: 2019.2.0.dev0
*** Git changeset:  1c52e837eb54cc34627f90bde254be4aa8a2ae17
*** -------------------------------------------------------------------------


Una vez que tenemos la solución, podemos calcular la tasa de flujo de calor por unidad de longitud así la comparamos con el resultado teórico. Para esto, primero debemos seleccionar la superficie (en este caso borde) donde queremos estimar el flujo, en este caso está etiquetada con 20. Note que también tenemos que conseguir la dirección normal para poder calcular lo siguiente:

$$\frac{q}{L} = -\int_{\partial \Omega_{2}} k\nabla T \cdot \overrightarrow{n} \ \ ds $$

In [ ]:
#Cálculo del flujo
ds = Measure('ds', domain=mesh, subdomain_data=boundary_markers)
n = FacetNormal(mesh)
flux = -(kappa)*dot(grad(u),n)*ds(20)
total_flux = assemble(flux)
print('Flujo numerico:',total_flux)

Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling form ffc_form_3b855e44c04a2f38c973c292571314cb37fa57ad

INFO:FFC:Compiler stage 1: Analyzing form(s)
INFO:FFC:-----------------------------------
DEBUG:FFC:  Preprocessing form using 'uflacs' representation family.
INFO:UFL_LEGACY:Adjusting missing element cell to triangle.
INFO:FFC:  
INFO:FFC:  Geometric dimension:                 2
  Number of exterior_facet subdomains: 21
  Rank:                                0
  Arguments:                           '()'
  Number of coefficients:              2
  Coefficients:                        '[f_13, f_14]'
  Unique elements:                     'CG2(?,?), Vector<2 x CG1(?,?)>'
  Unique sub elements:                 'CG2(?,?), Vector<2 x CG1(?,?)>, CG1(?,?)'
  
INFO:FFC:  representation:    auto --> uflacs
INFO:FFC:  quadrature_rule:   auto --> default
INFO:FFC:  quadrature_degree: auto --> 3
INFO:FFC:  quadrature_degree: 3
INFO:FFC:  
INFO:FFC

Flujo numerico: 924.3469714906935


Vemos que la diferencia es muy poca, se puede reducir haciendo una malla más fina.

## Problema con conductividad dependiente de la temperatura

El definir la formulación variacional de la forma $F = 0$ nos permite también utilizar este código para resolver problemas en los que la conductividad térmica depende de la temperatura. Es decir:

$$ \left \{ \begin{array}{l} -\left(\nabla \cdot k\left(T\right) \nabla T\right)=f \ \ \text{ para } \ \ x\in \Omega \\   T= 314,15 K  \ \ \text{condición de Dirichlet radio interno}  \ \ \partial \Omega_{1}\\   T= 310,15 K  \ \ \text{condición de Dirichlet radio externo} \ \ \partial \Omega_{2} \end{array} \right.$$

Ahora la conductividad dependerá de la posición y de la temperatura, es decir, la misma función que quiero computar. Por ejemplo, un función de temperatura que se utiliza mucho es la aproximación lineal en un rango no muy lejano de la temperatura de funcionamiento (digamos $T_{0}$). Supongamos que ésta aumenta el 2 % con el cambio de temperatura, es decir:

$$ k = k_{0}+\alpha k_{0} \left( T-T_{0} \right) = k_{0}\left( 1+\alpha \left( T-T_{0}\right) \right)$$

donde $\alpha = 0,02 \ \ K^{-1}$ y $k_{0}$ es la conductividad térmica a la temperatura $T_{0}$. En este tipo de problemas no voy a poder llegar a una expresión del tipo: $A\xi = b$. Por lo tanto, primero se **linealiza** y luego se resuelve **iterativamente**. En el medio, tiene que estimar el $F^{'}$. Por ahora no vamos a profundizar mucho en esto, solo diremos que  utiliza un algoritmo denominado **método inexacto de Newton** para resolverlo. Simplemente vamos a expresar la formulación variacional como sigue:

In [ ]:
F = kappa*(1+alfa*(u-Constant(310.15)))*dot(grad(u), grad(v))*dx-f*v*dx

NameError: name 'alfa' is not defined

Esto último está resuelto en el *ejemplo 12.py*. Más adelante hablaremos de las soluciones de sistemas lineales y no lineales del tipo a los que aparecen con elementos fintos.